Run this section first to load all the prerequisite packages.

In [115]:
from astroquery.vizier import Vizier
import requests
from bs4 import BeautifulSoup
from astroquery.simbad import Simbad
import numpy as np
from astropy.table import Table
from astroquery.xmatch import XMatch

Please run this section of the code to download the data on constellation borders. The csv file will contain the ra and dec that must be plotted (scatter) to obtain the borders of constellations

In [116]:
#First we import astroquery and remove the limit on rows.

from astroquery.vizier import Vizier
catalogue_list = Vizier.find_catalogs("Constellation")#This loads all the catalogs by the keyword "constellation"
Vizier.ROW_LIMIT = -1

catalog = Vizier.get_catalogs("VI/49")#This selects the 2nd of three tables that were found by the key-word search. This table contains relevant data
catalog[1].write("constellation_borders.csv",format="csv",overwrite="True")#creates a csv file

The output file generated by the above script will be stored in your working directory. to load tables directly from the Internet using URLs. For example, download tables from Vizier catalogues in CDS format ('ascii.cds'):

Run this section to download the ra,dec of messier objects

In [117]:
#PARSING THROUGH HYPERLEDA
#will add all mags into an array
mags = []

import requests
from bs4 import BeautifulSoup
for i in range(110):
    page = requests.get("http://leda.univ-lyon1.fr/ledacat.cgi?o=M{}".format(i+1))
    soup = BeautifulSoup(page.content,'html.parser')
    if len(soup.find_all('table',attrs={"class":"datatable"}))!=0:
        table_rel = soup.find_all('table',attrs={"class":"datatable"})[1]
        index = 0
        for rows in table_rel.find_all('tr'):
            if rows.find_all('a', attrs = {"title":"Total V-magnitude"}) != []:
                            mag_value = rows.find_all('td')[1].get_text().split()[0]
                            mags.append(mag_value)
                            index = index + 1
                            break
            if rows == table_rel.find_all('tr')[-1] and index == 0:
                mags.append('-')
                break
    else:
        mags.append('-')
        continue

#NEED TO ADD THESE VALUES TO THE MESSIER TABLE

In [118]:

#Import Simbad query
from astroquery.simbad import Simbad

#the next three lines are to simplify the output table
Simbad.reset_votable_fields()
Simbad.remove_votable_fields('coordinates')
Simbad.add_votable_fields('otype(S)', 'ra(d;A;ICRS;J2000;2000)', 'dec(d;D;ICRS;J2000;2000)','distance','flux(B)','flux(V)','flux_unit(V)')

result_table = Simbad.query_catalog("Messier")#This asks the SIMBAD database to list all objects from the messier catalog
result_table['V Mag (from HyperLeda/SED)'] = mags#adding v mags
result_table.write("messier_objects.csv",format="csv",overwrite="True")#creates a csv file

Simbad.reset_votable_fields()#renders the prev changes to simbad class temporary.

NGC Catalogue

In [119]:

from astroquery.vizier import Vizier
v = Vizier(columns = ['Name','Type','mag','Const','RA (deg)','Dec (deg)'])
v.ROW_LIMIT = -1
result_table = v.get_catalogs("VII/118/ngc2000")[0]
result_table.write("NGC.csv",format="csv",overwrite="True")

Hipparcos Catalogue

In [120]:
from astroquery.vizier import Vizier
v = Vizier(columns = ['HD','_RAJ2000','DECJ2000','B-V','Vmag','SpType','Plx'], column_filters = {"Vmag":"<=6"})
v.ROW_LIMIT = -1
v.TIMEOUT = 1000
catalog_stars = v.get_catalogs("I/239/hip_main")[0]
catalog_stars.write("Hipparcos_1.csv",format="csv",overwrite = "True")


In [121]:
from astroquery.vizier import Vizier
v = Vizier(columns = ['HD','_RAJ2000','DECJ2000','B-V','Vmag','SpType','Plx'], column_filters = {"Vmag":"6..8"})
v.ROW_LIMIT = -1
v.TIMEOUT = 1000
catalog_stars = v.get_catalogs("I/239/hip_main")[0]
catalog_stars.write("Hipparcos_2.csv",format="csv",overwrite = "True")

NOMAD Catalogue

In [122]:
from astroquery.vizier import Vizier
v = Vizier(column_filters = {"Vmag":"<=6.00"})
v.ROW_LIMIT = -1
v.TIMEOUT = 1000
catalog_strs = v.get_catalogs("I/297/out")[0]
catalog_strs.write("NOMAD_1.csv", format = "csv", overwrite = "True")

In [123]:
v = Vizier(column_filters = {"Vmag":"6.0..8.0"})
v.ROW_LIMIT=-1
v.TIMEOUT = 1000
catalog_strs = v.get_catalogs("I/297/out")[0]
catalog_strs.write("NOMAD_2.csv", format = "csv", overwrite = "True")

In [124]:
v = Vizier(column_filters = {"Vmag":"8.0..10.0"})
v.ROW_LIMIT=-1
v.TIMEOUT = 1000
catalog_strs = v.get_catalogs("I/297/out")[0]
catalog_strs.write("NOMAD_3.csv", format = "csv", overwrite = "True")

Can stop for the stars here. This catalog is very heavily populated from 10 mag onwards.

In [ ]:
v = Vizier(column_filters = {"Vmag":"10.0..11.0"})
v.ROW_LIMIT=-1
v.TIMEOUT = 1000
catalog_strs = v.get_catalogs("I/297/out")[0]
catalog_strs.write("NOMAD_4.csv", format = "csv", overwrite = "True")

If you want to cross match the hipparcos and nomad files, you can use the XMatch module.

In [24]:
from astroquery.xmatch import XMatch
from astropy import units as u
#print(XMatch.is_table_available("I/239/hip_main"))

table = XMatch.query(cat1=open('NOMAD_1.csv'), cat2="vizier:I/239/hip_main", max_distance=18 * u.arcsec, colRA1='RAJ2000', colDec1='DEJ2000')
table.write("NOMAD_1_join.csv",format='csv', overwrite = 'True')

In [ ]:
from astropy.table import Table
table = Table.read('NOMAD_1_join.csv')
table2 = Table.read('NOMAD_1.csv')
ra1 = table['RAJ2000']
ra2 = table2['RAJ2000']
count = 0
for elt in ra2:
    print(elt in ra1)

Bright Star Catalogue

In [126]:
from astroquery.vizier import Vizier
catalog_strs = Vizier.get_catalogs("V/50")
Vizier.ROW_LIMIT = -1
catalog_strs[0].write("BSC5P.csv",format="csv",overwrite="True")

Adding Star names to the catalogs using their HR names

In [ ]:
import numpy as np

name_data = np.loadtxt("IAU-CSN.csv",delimiter=",", usecols=(1,3,4,9), dtype = np.str)
names = name_data[1:,0]
idc = name_data[1:,1]
cons = name_data[1:,2]
hdno = name_data[1:,3]

star_file = np.loadtxt("hipparcos.csv",delimiter=",", usecols=(1), dtype=np.str)
hd = star_file[1:]

common = np.where([x in hdno for x in hd])[0]
names_lst = np.array(['-' for x in hd])
names_lst[common] = names
        

Guide Star Catalogue